In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,6182.01,-0.17,-0.17,-0.21,0.24,0.15,0.03,0.00,0.05,0.09,...,-0.00,-0.01,0.02,0.00,0.01,-0.02,0.00,0.01,-0.00,0.00
2,-0.17,8192.01,-0.05,0.36,-0.10,0.10,-0.18,0.00,0.11,0.07,...,0.02,0.02,0.01,-0.01,0.02,0.04,0.01,0.01,0.00,0.00
3,-0.17,-0.05,3442.39,-0.11,-0.00,0.05,0.08,-0.01,-0.01,0.04,...,-0.01,-0.00,0.03,0.01,-0.00,0.01,-0.00,-0.00,-0.00,-0.00
4,-0.21,0.36,-0.11,8372.48,0.06,0.17,0.12,0.01,0.10,0.06,...,-0.01,0.00,-0.00,-0.02,0.01,-0.01,0.00,-0.00,0.00,-0.00
5,0.24,-0.10,-0.00,0.06,3458.32,0.03,-0.06,0.00,-0.00,0.14,...,0.01,0.00,-0.00,-0.00,0.01,0.01,-0.00,0.00,-0.00,-0.00
6,0.15,0.10,0.05,0.17,0.03,2964.91,-0.10,-0.01,-0.19,-0.06,...,-0.01,0.02,0.00,0.00,-0.01,0.02,-0.00,-0.00,0.00,-0.00
7,0.03,-0.18,0.08,0.12,-0.06,-0.10,3957.06,0.01,-0.01,0.00,...,0.01,-0.00,0.02,-0.00,-0.02,0.01,0.00,-0.00,0.00,-0.00
8,0.00,0.00,-0.01,0.01,0.00,-0.01,0.01,125.89,-0.00,0.04,...,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00
9,0.05,0.11,-0.01,0.10,-0.00,-0.19,-0.01,-0.00,1136.19,0.06,...,0.00,0.00,0.02,0.01,0.00,0.01,-0.00,-0.00,-0.00,-0.00
10,0.09,0.07,0.04,0.06,0.14,-0.06,0.00,0.04,0.06,2741.77,...,0.00,-0.00,-0.02,0.00,0.01,-0.01,-0.00,-0.00,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,-0.00002,-0.00004,-0.00003,0.00005,0.00004,0.00001,0.00000,0.00002,0.00002,...,-0.00000,-0.00001,0.00001,0.00000,0.00001,-0.00001,0.00001,0.00001,-0.00000,0.00000
2,-0.00002,1.00000,-0.00001,0.00004,-0.00002,0.00002,-0.00003,0.00000,0.00003,0.00001,...,0.00002,0.00001,0.00001,-0.00000,0.00001,0.00002,0.00001,0.00001,0.00000,0.00000
3,-0.00004,-0.00001,1.00000,-0.00002,-0.00000,0.00002,0.00002,-0.00001,-0.00001,0.00001,...,-0.00001,-0.00000,0.00002,0.00001,-0.00000,0.00001,-0.00001,-0.00000,-0.00000,-0.00000
4,-0.00003,0.00004,-0.00002,1.00000,0.00001,0.00003,0.00002,0.00001,0.00003,0.00001,...,-0.00001,0.00000,-0.00000,-0.00001,0.00000,-0.00000,0.00001,-0.00000,0.00000,-0.00001
5,0.00005,-0.00002,-0.00000,0.00001,1.00000,0.00001,-0.00002,0.00000,-0.00000,0.00005,...,0.00001,0.00000,-0.00000,-0.00000,0.00001,0.00001,-0.00000,0.00000,-0.00000,-0.00001
6,0.00004,0.00002,0.00002,0.00003,0.00001,1.00000,-0.00003,-0.00002,-0.00010,-0.00002,...,-0.00001,0.00002,0.00000,0.00000,-0.00001,0.00002,-0.00000,-0.00001,0.00001,-0.00001
7,0.00001,-0.00003,0.00002,0.00002,-0.00002,-0.00003,1.00000,0.00001,-0.00001,0.00000,...,0.00001,-0.00000,0.00001,-0.00000,-0.00002,0.00001,0.00001,-0.00000,0.00000,-0.00001
8,0.00000,0.00000,-0.00001,0.00001,0.00000,-0.00002,0.00001,1.00000,-0.00001,0.00006,...,-0.00003,-0.00002,-0.00001,0.00001,0.00001,-0.00000,0.00000,-0.00000,-0.00000,0.00000
9,0.00002,0.00003,-0.00001,0.00003,-0.00000,-0.00010,-0.00001,-0.00001,1.00000,0.00003,...,0.00000,0.00000,0.00002,0.00001,0.00000,0.00002,-0.00001,-0.00000,-0.00001,-0.00000
10,0.00002,0.00001,0.00001,0.00001,0.00005,-0.00002,0.00000,0.00006,0.00003,1.00000,...,0.00000,-0.00000,-0.00002,0.00000,0.00001,-0.00001,-0.00001,-0.00000,-0.00001,0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

4.1202262158743155

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.44629952e+06 1.26224171e+06 1.18188800e+06 1.15644664e+06
 1.09878067e+06 6.50343435e+05 6.28566783e+05 4.62227820e+05
 3.33408543e+05 2.61653905e+05 2.02947506e+05 2.01672590e+05
 1.95632774e+05 1.83412745e+05 1.56526294e+05 1.46854751e+05
 1.33790155e+05 1.28846369e+05 1.24691605e+05 1.16596947e+05
 1.11964478e+05 1.04150414e+05 9.95345480e+04 9.18721936e+04
 8.94047024e+04 8.48544791e+04 7.79946025e+04 7.73394288e+04
 7.70815284e+04 7.54818120e+04 7.44849525e+04 6.22328810e+04
 5.20679421e+04 4.54646437e+04 4.07942041e+04 3.41683150e+04
 2.83855224e+04 2.17090542e+04 1.98271525e+04 1.42011143e+04
 1.07529517e+04 1.03776347e+04 9.86670561e+03 9.61499276e+03
 9.33813595e+03 7.91386804e+03 6.52431350e+03 5.34762475e+03
 4.69215856e+03 3.97116382e+03 3.20761239e+03 2.35555678e+03
 1.61887318e+03 9.63699232e+02 5.49208692e+02 2.58110181e+02
 2.35154837e+02 8.78615492e+01 7.30051310e+00 5.95541734e-25]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,-0.008395,-0.013595,-0.004046,-0.014167,-0.004044,-0.003430,-0.004745,-0.000141,-0.001271,-0.003146,...,-0.000218,-0.000441,-0.000720,-0.000383,-0.000284,-0.000542,-0.000026,-0.000129,-0.000133,-0.000021
1,-0.013959,-0.091499,-0.004496,-0.157877,-0.004559,-0.003728,-0.005548,-0.000139,-0.001279,-0.003377,...,-0.000219,-0.000446,-0.000717,-0.000383,-0.000291,-0.000544,-0.000025,-0.000130,-0.000131,-0.000021
2,-0.007302,-0.113393,-0.002242,0.848640,-0.002209,-0.001786,-0.002679,-0.000066,-0.000599,-0.001642,...,-0.000106,-0.000213,-0.000349,-0.000185,-0.000139,-0.000262,-0.000012,-0.000062,-0.000063,-0.000010
3,-0.016312,-0.576864,-0.004789,0.355668,-0.004835,-0.003928,-0.005921,-0.000146,-0.001352,-0.003547,...,-0.000233,-0.000466,-0.000752,-0.000396,-0.000301,-0.000575,-0.000027,-0.000136,-0.000138,-0.000022
4,-0.052005,0.801575,-0.014495,0.353848,-0.014617,-0.011776,-0.017998,-0.000433,-0.003979,-0.010652,...,-0.000683,-0.001378,-0.002242,-0.001179,-0.000892,-0.001694,-0.000077,-0.000401,-0.000410,-0.000064
5,-0.629586,0.004274,-0.001926,0.003910,-0.002074,-0.001616,-0.002703,-0.000051,-0.000485,-0.001394,...,-0.000079,-0.000161,-0.000264,-0.000139,-0.000107,-0.000202,-0.000011,-0.000048,-0.000048,-0.000007
6,0.771107,0.038500,-0.019251,0.035773,-0.019335,-0.014764,-0.026032,-0.000478,-0.004470,-0.013057,...,-0.000755,-0.001529,-0.002488,-0.001303,-0.000987,-0.001886,-0.000088,-0.000443,-0.000453,-0.000071
7,0.050334,0.018226,-0.019922,0.017249,-0.020151,-0.014292,-0.030356,-0.000418,-0.003902,-0.012353,...,-0.000655,-0.001314,-0.002156,-0.001120,-0.000854,-0.001616,-0.000074,-0.000381,-0.000388,-0.000061
8,0.022558,0.011712,-0.029803,0.011367,-0.030330,-0.018564,-0.067242,-0.000434,-0.004202,-0.015228,...,-0.000685,-0.001388,-0.002288,-0.001185,-0.000901,-0.001717,-0.000078,-0.000403,-0.000414,-0.000064
9,0.014976,0.008799,-0.052801,0.008436,-0.055038,-0.024739,0.989908,-0.000449,-0.004437,-0.019093,...,-0.000709,-0.001445,-0.002378,-0.001231,-0.000937,-0.001781,-0.000081,-0.000418,-0.000426,-0.000067


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.008395140057986,
 1.091499031349198,
 1.0022416331175927,
 0.6443315634528344,
 1.014616778923542,
 1.0016162922408907,
 1.0260323891955918,
 1.000418103511864,
 1.004201562726504,
 1.01909312559458,
 1.0011637188951463,
 0.9970268891231676,
 1.0407737706936546,
 0.806724351129403,
 0.043386582095915616,
 1.0131932435653195,
 1.0195621897794718,
 1.0038340901389207,
 0.988224097424258,
 0.9969038435426457,
 0.9966399895680614,
 1.0348350757323461,
 0.9858701598369747,
 0.992680132054257,
 1.0072012399226487,
 1.0257893465788916,
 1.0009277670848142,
 0.9993037603935214,
 1.0034899821770935,
 0.530484808507,
 1.0398755113390723,
 1.0930000593004647,
 1.013388472967938,
 0.9522756060045711,
 0.9871412984093143,
 0.992584572341117,
 0.9612921590864022,
 0.9956194521386058,
 0.9042710884875668,
 1.0856087764474411,
 0.9971395843646886,
 0.6647972069542353,
 0.996406990576954,
 1.001084739163852,
 0.9822645079198117,
 1.2711627497257467,
 1.1230572310742435,
 0.99400595913812,
 0.9217923